In [ ]:
log_path = 'fixed_value/2025-10-21'
import numpy as np
import re
import os
from omegaconf import OmegaConf

def get_metrics(data):
    overall_distance_p = re.compile(r'Overall consensus distance : ([\d\.]+)')
    inter_cluster_distance_p = re.compile(r'Inter-cluster distance : ([\d\.]+)')
    cluster_1_distance_p =  re.compile(r'Inter-cluster distance : ([\d\.]+)')
    cluster_2_distance_p =  re.compile(r'Cluster 2 consensus distance : ([\d\.]+)')
    
    overall_distance, inter_cluster_distance, cluster_1_distance, cluster_2_distance = [], [], [], []
    for match in overall_distance_p.finditer(data):
        overall_distance.append(float(match.group(1)))
        
    for match in inter_cluster_distance_p.finditer(data):
        inter_cluster_distance.append(float(match.group(1)))
    
    for match in cluster_1_distance_p.finditer(data):
        cluster_1_distance.append(float(match.group(1)))
           
    for match in cluster_2_distance_p.finditer(data):
        cluster_2_distance.append(float(match.group(1)))
        
    return overall_distance, inter_cluster_distance, cluster_1_distance, cluster_2_distance

res = {}
for log_dir in os.listdir(log_path):
    log_file  = f'{log_path}/{log_dir}'
    if os.path.isfile(log_file):
        continue
    with open(f'{log_file}/main.log', 'r') as f:
        data = f.read()
    config_file = OmegaConf.load(f'{log_file}/.hydra/config.yaml') 
    overall_distance, inter_cluster_distance, cluster_1_distance, cluster_2_distance = get_metrics(data)
    res[log_dir.split('/')[-1]] = [overall_distance, inter_cluster_distance, cluster_1_distance, cluster_2_distance ]
    


FileNotFoundError: [Errno 2] No such file or directory: 'fixed_value/2025-10-21/13-59-07/main.log'